## Shadie demo (v.0.1.1)

In [1]:
import numpy as np
import shadie
import toytree

print("shadie", shadie.__version__)
print("toytree", toytree.__version__)

shadie 0.1.1
toytree 2.1.1-dev


### Create a simple chromosome

In [2]:
chrom = shadie.chromosome.explicit({
    (0, 1_200_000): shadie.NONCDS,
    (1_200_001, 1_400_000): shadie.EXON,
    (1_400_001, 1_600_000): shadie.INTRON,
    (1_600_001, 1_800_000): shadie.EXON,
    (1_800_001, 5_000_000): shadie.NONCDS,
})
chrom.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t535813942bd1452392b1414fba86426e"> name: noncds
interval: (0, 1200000)
ElementType: g3
coding: False name: exon
interval: (1200001, 1400000)
ElementType: g1
coding: True name: intron
interval: (1400001, 1600000)
ElementType: g2
coding: True name: exon
interval: (1600001, 1800000)
ElementType: g1
coding: True name: noncds
interval: (1800001, 5000000)
ElementType: g3
coding: False 0 1000000 2000000 3000000 4000000 5000000

### Setup models

In [3]:
with shadie.Model() as model_1:
    model_1.initialize(
        chromosome=chrom, 
        generations=5000,
        mut=1e-8, 
        recomb=1e-8,
        trees_file="/tmp/test-2.trees",
    )
    model_1.reproduction.bryophyte("dio", 5000, 5000)

In [4]:
with shadie.Model() as model_2:
    model_2.initialize(
        chromosome=chrom, 
        generations=5000,
        mut=1e-8, 
        recomb=1e-8,
        trees_file="/tmp/test-3.trees",
    )
    model_2.reproduction.bryophyte("dio", 5000, 5000)

### Run two populations in parallel

Each simulation model is run from a different random seed, and writes to a different .trees file path. I also provide the path to my updated `slim` binary as an argument.

In [5]:
from concurrent.futures import ProcessPoolExecutor

In [6]:
with ProcessPoolExecutor(2) as pool:
    for model in [model_1, model_2]:
        kwargs = {"binary": "/usr/local/bin/slim", "seed": np.random.randint(2**31)}
        pool.submit(model.run, **kwargs)

### Get post-processor

In [7]:
post = shadie.TwoSims(
    tree_files=["/tmp/test-2.trees", "/tmp/test-3.trees"],
    mut=1e-8,
    recomb=1e-8,
    popsize=5000,
    chromosome=chrom,
)

05:13 | INFO    | _report_ninds   | inds alive at time=0; simpop0=5199, simpop1=5447
05:13 | INFO    | _report_mutations | Keeping 149 existing mutations of type(s) {2, 3}.
05:13 | INFO    | _report_mutations | Keeping 8524 existing mutations of type(s) {0, 2, 3}.


### Plot simulation summary

In [10]:
post.draw_tree_sequence(sample=6, seed=333);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1800.0px" height="425.0px" viewBox="0 0 1800.0 425.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t569e61e2519541069631863048d29080"> name: noncds
interval: (0, 1200000)
ElementType: g3
coding: False name: exon
interval: (1200001, 1400000)
ElementType: g1
coding: True name: intron
interval: (1400001, 1600000)
ElementType: g2
coding: True name: exon
interval: (1600001, 1800000)
ElementType: g1
coding: True name: noncds
interval: (1800001, 5000000)
ElementType: g3
coding: False 0 1000000 2000000 3000000 4000000 5000000 <polygon points="50.0,108.49385073285787 59.4896331738437,142.46925366428934 59.4896331738437,312.34626832144664 227.59170653907498,312.34626832144664 227.59170653907498,142.46925366428934 296.89820495622376,108.49385073285787 296.89820495622376,108.49385073285787 227.59170653907498,108.49385073285787 227.59170653907498,108.49385073285787 59.4896331738437,108.49385073285787 59.4896331738437,108.49385073285787 50.0,108.49385073285787" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 0
interval: (0 - 40033)
tmrca: 28985.15 <polygon points="296.89820495622376,108.49385073285787 227.59170653907498,142.46925366428934 227.59170653907498,312.34626832144664 395.6937799043062,312.34626832144664 395.6937799043062,142.46925366428934 479.89015258083623,108.49385073285787 479.89015258083623,108.49385073285787 395.6937799043062,108.49385073285787 395.6937799043062,108.49385073285787 227.59170653907498,108.49385073285787 227.59170653907498,108.49385073285787 296.89820495622376,108.49385073285787" style="fill:rgb(98.8%,55.3%,38.4%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 1
interval: (40033 - 69704)
tmrca: 28985.15 <polygon points="479.89015258083623,108.49385073285787 395.6937799043062,142.46925366428934 395.6937799043062,312.34626832144664 563.7958532695375,312.34626832144664 563.7958532695375,142.46925366428934 632.9210310955311,108.49385073285787 632.9210310955311,108.49385073285787 563.7958532695375,108.49385073285787 563.7958532695375,108.49385073285787 395.6937799043062,108.49385073285787 395.6937799043062,108.49385073285787 479.89015258083623,108.49385073285787" style="fill:rgb(55.3%,62.7%,79.6%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 2
interval: (69704 - 94517)
tmrca: 28985.15 <polygon points="632.9210310955311,108.49385073285787 563.7958532695375,142.46925366428934 563.7958532695375,312.34626832144664 731.8979266347689,312.34626832144664 731.8979266347689,142.46925366428934 886.4923271956269,108.49385073285787 886.4923271956269,108.49385073285787 731.8979266347689,108.49385073285787 731.8979266347689,108.49385073285787 563.7958532695375,108.49385073285787 563.7958532695375,108.49385073285787 632.9210310955311,108.49385073285787" style="fill:rgb(90.6%,54.1%,76.5%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 3
interval: (94517 - 135632)
tmrca: 28985.15 <polygon points="886.4923271956269,108.49385073285787 731.8979266347689,142.46925366428934 731.8979266347689,312.34626832144664 900.0,312.34626832144664 900.0,142.46925366428934 1074.449005292344,108.49385073285787 1074.449005292344,108.49385073285787 900.0,108.49385073285787 900.0,108.49385073285787 731.8979266347689,108.49385073285787 731.8979266347689,108.49385073285787 886.4923271956269,108.49385073285787" style="fill:rgb(65.1%,84.7%,32.9%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 4
interval: (135632 - 166108)
tmrca: 28985.15 idx: 5


### Plot individual trees

In [11]:
post.draw_tree(idx=0, sample=[10, 10], seed=123);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="400.0px" viewBox="0 0 400.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1828f92cbbac4bf29175a2e648fb004c"> p1-14 p1-13 p1-16 p1-11 p1-17 p1-10 p1-19 p1-18 p1-15 p1-12 p0-7 p0-3 p0-6 p0-0 p0-1 p0-5 p0-9 p0-4 p0-8 p0-2 id: 0
pos: 1063
time: 13927
mtype: 0 id: 1
pos: 1319
time: 4993
mtype: 0 id: 2
pos: 1705
time: 18154
mtype: 0 id: 3
pos: 3577
time: 9520
mtype: 0 id: 4
pos: 4031
time: 1881
mtype: 0 id: 5
pos: 4498
time: 353
mtype: 0 id: 6
pos: 5214
time: 14855
mtype: 0 id: 7
pos: 5277
time: 14263
mtype: 0 id: 8
pos: 5445
time: 7204
mtype: 0 id: 9
pos: 8071
time: 140
mtype: 0 id: 10
pos: 8651
time: 7288
mtype: 0 id: 11
pos: 8732
time: 20318
mtype: 0 id: 12
pos: 11482
time: 22504
mtype: 0 id: 13
pos: 13447
time: 17161
mtype: 0 id: 14
pos: 15907
time: 77
mtype: 0 id: 15
pos: 16883
time: 15071
mtype: 0 id: 16
pos: 17259
time: 7064
mtype: 0 id: 17
pos: 20070
time: 17905
mtype: 0 id: 18
pos: 20537
time: 3601
mtype: 0 id: 19
pos: 21112
time: 1075
mtype: 0 id: 20
pos: 21994
time: 26993
mtype: 0 id: 21
pos: 23109
time: 8011
mtype: 0 id: 22
pos: 24723
time: 27973
mtype: 0 id: 23
pos: 25420
time: 11523
mtype: 0 id: 24
pos: 27523
time: 16953
mtype: 0 id: 25
pos: 28368
time: 13470
mtype: 0 id: 26
pos: 32536
time: 28948
mtype: 0 id: 27
pos: 33583
time: 26847
mtype: 0 id: 28
pos: 39087
time: 11924
mtype: 0 id: 29
pos: 39122
time: 16485
mtype: 0 id: 30
pos: 39428
time: 84
mtype: 0 id: 31
pos: 39609
time: 28596
mtype: 0 id: 32
pos: 39892
time: 15537
mtype: 0 25000 20000 15000 10000 5000 0 tree in interval 0 (sites 0-40033)

### Calculate statistics

In [12]:
post.stats(sample=10, reps=20)

,mean,CI_5%,CI_95%
theta_0,0.000012,0.000011,0.000012
theta_1,0.000014,0.000014,0.000015
Fst_01,0.958149,0.957169,0.959129
Dist_01,0.000608,0.000607,0.000608
D_Taj_0,-0.919146,-0.994521,-0.843772
D_Taj_1,-0.648495,-0.740346,-0.556643
